## Семинар 1 Индекс

## Intro

### работа с файлами и папками

In [1]:
import os

curr_dir = os.getcwd()
filepath = os.path.join(curr_dir, 'test.txt')

### os.path  
путь до файла

In [2]:
# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath(filepath))


# возвращает имя файла / папки по полному ти до него
print(os.path.basename(filepath))


# проверить существование директории - True / False
print(os.path.exists(curr_dir))

/Users/victoriaregina/infosearch/1 Index/test.txt
test.txt
True


### os.listdir  
возвращает список файлов в данной директории

In [3]:
os.listdir(curr_dir)

['.ipynb_checkpoints', 'lec1_Index.pdf', 'sem1_Index.ipynb', 'test.txt']

При обходе файлов не забывайте исключать системные директории, такие как .DS_Store

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [4]:
for root, dirs, files in os.walk(curr_dir):
    for name in files:
        print(os.path.join(root, name))

/Users/victoriaregina/infosearch/1 Index/lec1_Index.pdf
/Users/victoriaregina/infosearch/1 Index/sem1_Index.ipynb
/Users/victoriaregina/infosearch/1 Index/test.txt
/Users/victoriaregina/infosearch/1 Index/.ipynb_checkpoints/sem1_Index-checkpoint.ipynb


> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [5]:
list(os.walk(curr_dir))

[('/Users/victoriaregina/infosearch/1 Index',
  ['.ipynb_checkpoints'],
  ['lec1_Index.pdf', 'sem1_Index.ipynb', 'test.txt']),
 ('/Users/victoriaregina/infosearch/1 Index/.ipynb_checkpoints',
  [],
  ['sem1_Index-checkpoint.ipynb'])]

### чтение файла 

In [6]:
fpath = 'test.txt'


# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

    
#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

    
#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 

Напоминание про enumerate:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, не надо так

##  Индекс 

Сам по себе индекс - это просто формат хранения данных, он не может осуществлять поиск. Для этого необходимо добавить к нему определенную метрику. Это может быть что-то простое типа булева поиска, а может быть что-то более специфическое или кастомное под задачу.

Давайте посмотрим, что полезного можно вытащить из самого индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе - так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. как эти документы кластеризуются по N тематикам согласно словам, которые в них упоминаются 

## __Задача__: 

**Data:** Коллекция субтитров сезонов Друзьей. Одна серия - один документ.

**To do:** Постройте небольшой модуль поискового движка, который сможет осуществлять поиск по коллекции документов.
На входе запрос и проиндексированная коллекция (в том виде, как посчитаете нужным), на выходе отсортированный по релевантности с запросом список документов коллекции. 

Релизуйте:
    - функцию препроцессинга данных
    - функцию индексирования данных
    - функцию метрики релевантности 
    - собственно, функцию поиска

[download_friends_corpus](https://yadi.sk/d/yVO1QV98CDibpw)

Напоминание про defaultdict: 
> В качестве multiple values словаря рекомендую использовать ``` collections.defaultdict ```                          
> Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [327]:
import os
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import nltk
import collections
import re
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [328]:
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/victoriaregina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [330]:
from collections import defaultdict

corpus = []
ep_per_seasons = {}
names_docs = []

main_dir = '/Users/victoriaregina/Downloads/friends'
folders = [f for f in os.listdir(main_dir) if 'Store' not in f]


def preprocessing(text):
    normal_forms = []
    tokenizer = RegexpTokenizer(r'\w+')
    
    tokens = [morph.parse(token)[0] for token in tokenizer.tokenize(text)]
    normal_forms = [token.normal_form for token in tokens if token not in russian_stopwords and token.normal_form != 'это']
    preproc_text = ' '.join(normal_forms)
            
    return preproc_text
                   
    
for folder in folders:
    for file in os.listdir(os.path.join(main_dir, folder)):
        filepath = os.path.join(main_dir, folder, file)
        names_docs.append(file)
        
        with open(filepath, 'r') as f:
            text = f.read()
        
        preproc_text = preprocessing(text)
        corpus.append(preproc_text)
        
        ep_per_seasons[file] = folder

In [331]:
print(len(names_docs))
print(len(ep_per_seasons))

165
165


In [332]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()
X = vec.fit_transform(corpus)

df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names(), index=names_docs)

In [333]:
df.shape

(165, 15048)

In [334]:
def search(query):
    query_preproc = preprocessing(query)
    vec_query = vec.transform([query_preproc])
    
    vec_query = vec.transform([query_preproc])

    results = {}

    for index, row in df.iterrows():
        vector = row.as_matrix()

        cos_sim = cosine_similarity(vector.reshape(1, -1), vec_query)
        cos_sim = np.asscalar(cos_sim)

        results[cos_sim] = index


    return results

In [335]:
query = input('Введите запрос: ')

results = search(query)

print('Вот 10 самых подходящих серий: ')

for i, key in enumerate(sorted(results, reverse=True)[:10]):
    print(str(i+1) + ': ' + results[key])

Введите запрос: привет росс
Вот 10 самых подходящих серий: 
1: Friends - 5x20 - The One With The Ride Along.ru.txt
2: Friends - 5x01 - The One After Ross Says Rachel.ru.txt
3: Friends - 7x04 - The One With Rachel's Assistant.ru.txt
4: Friends - 2x01 - The One With Ross's New Girlfriend.DVDRip.ru.txt
5: Friends - 4x24-25 - The One With Ross's Wedding (2).ru.txt
6: Friends - 6x21 - The One Where Ross Meets Elizabeth's Dad.ru.txt
7: Friends - 5x06 - The One With The Yeti.ru.txt
8: Friends - 4x23-24 - The One With Ross's Wedding (1).ru.txt
9: Friends - 2x01 - The One With Ross's New Girlfriend.ru.txt
10: Friends - 5x02 - The One With All The Kissing.ru.txt


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  del sys.path[0]


# Задания (a) и (b)

In [336]:
dic = {}

for c in vec.get_feature_names():
    dic[sum(df[c])] = c
    
print('Самое частотное слово:', dic[max(dic.keys())], max(dic.keys()))
print('Самое редкое слово:', dic[min(dic.keys())], min(dic.keys()))

Самое частотное слово: ты 56.81246170527751
Самое редкое слово: элегантный 0.01602731028921031


# Задание (c)

In [337]:
in_all_docs = []
d = df.isin([0])

for c in vec.get_feature_names():
    if sum(d[c]) == 0:
        in_all_docs.append(c)
        
print('Слова, которые есть во всех текстах:', ', '.join(in_all_docs))

Слова, которые есть во всех текстах: быть, весь, да, думать, если, ещё, знать, как, мой, мочь, мы, на, не, нет, но, ну, он, она, просто, сказать, так, такой, то, тот, ты, хотеть, что, этот


# Задание (d)

In [338]:
def popular_season_counting(df, character, seasons):
    
    data = df[character]
    row_names = data.index

    s1 = []
    s2 = []
    s3 = []
    s4 = []
    s5 = []
    s6 = []
    s7 = []
    
    count = {}

    for i, value in enumerate(data):
        if seasons[row_names[i]] == 'Friends - season 1':
            s1.append(value)
        elif seasons[row_names[i]] == 'Friends - season 2':
            s2.append(value)
        elif seasons[row_names[i]] == 'Friends - season 3':
            s3.append(value)
        elif seasons[row_names[i]] == 'Friends - season 4':
            s4.append(value)
        elif seasons[row_names[i]] == 'Friends - season 5':
            s5.append(value)
        elif seasons[row_names[i]] == 'Friends - season 6':
            s6.append(value)
        else:
            s7.append(value)
    keys = [numpy.sum(s1), numpy.sum(s2), numpy.sum(s3), numpy.sum(s4), numpy.sum(s5),
       numpy.sum(s6), numpy.sum(s7)]
    values = ['Friends - season 1', 'Friends - season 2', 'Friends - season 3', 'Friends - season 4',
             'Friends - season 5', 'Friends - season 6', 'Friends - season 7']
    count = dict(zip(keys, values))
    return count[max(count.keys())]  

In [339]:
print('Самый популярный сезон у Моники:', popular_season_counting(df, 'моника', ep_per_seasons))

Самый популярный сезон у Моники: Friends - season 7


In [340]:
print('Самый популярный сезон у Чендлера:', popular_season_counting(df, 'чендлер', ep_per_seasons))

Самый популярный сезон у Чендлера: Friends - season 6


# Задание (e)

In [341]:
d = {}
characters = ['росс', 'фиби', 'моника', 'чендлер', 'джо', 'рэйчел']

for character in characters:
    try:
        d[sum(df[character])] = character
    except KeyError:
        break

print('Самое популярный герой:', d[max(d.keys())], max(d.keys()))

Самое популярный герой: росс 5.320893775007355
